In [1]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [2]:
# from mlxtend.frequent_patterns import apriori

from surprise import Reader, Dataset, SVD, NormalPredictor
# from surprise import evaluate
import surprise.model_selection

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import scipy.stats
from sklearn import preprocessing
import sklearn.cluster
import keras
import keras.utils
from keras import optimizers, backend as K
from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


In [3]:
import util.plot
import util.data

In [4]:
reload(util.plot)
reload(util.data);

In [84]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=1*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)
data.shape, data.srch_id.unique().size

((1000, 134), 40)

In [88]:
# m = preprocessing.KBinsDiscretizer(n_bins=3, encode='onehot', strategy='quantile')
# x = pd.DataFrame({'A':np.arange(9), 'B':np.random.random(9)})
# z = x.A.values.reshape(-1,1)
# m.fit(z)
# y = m.transform(z)

In [89]:
# # type(np.random.random(2))
# isinstance(np.random.random(2), np.ndarray)
# pd.DataFrame(y)
# pd.SparseArray(y)
# y.toarray()

In [90]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
# varsUsed.remove('position')
# varsUsed.remove('random_bool')
len(varsUsed)

110

In [91]:
data.head()

,srch_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,position,price_usd,...,srch_booking_window_label0,srch_booking_window_label1,srch_booking_window_label2,srch_booking_window_label3,srch_booking_window_label4,prop_log_historical_price_bin0,prop_log_historical_price_bin1,prop_log_historical_price_bin2,prop_log_historical_price_bin3,prop_log_historical_price_bin4
0,1,0.0,0.0,893,-0.5,1,-0.025862,-0.344778,27,-0.047673,...,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,0.0,10404,0.0,1,-0.297414,-0.553668,26,0.578822,...,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0
2,1,0.0,0.0,21315,0.5,1,-0.297414,-0.484279,21,0.664862,...,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0
3,1,0.0,0.0,27348,0.0,1,-0.025862,-0.571016,34,4.681671,...,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,29604,-0.5,1,-0.107759,0.235634,4,0.320893,...,1,0,0,0,0,0.0,0.0,1.0,0.0,0.0


# Clustering

See also https://blog.keras.io/building-autoencoders-in-keras.html and https://github.com/voschezang/drum-style-transfer

In [92]:
# [k for k in data.columns if 'comp' not in k and 'prop' not in k]

In [93]:
# rm irrelevant info
keys_property = [k for k in data.columns if 'comp' in k or 'prop' in k]
keys_property += ['click_bool', 'booking_bool', 'gross_bookings_usd', 'random_bool', 'score', 'price_usd', 'position', 'travel_distance']
keys_search = [k for k in data.columns if k not in keys_property]
len(keys_search), keys_search

(74,
 ['srch_id',
  'visitor_hist_starrating',
  'visitor_hist_adr_usd',
  'promotion_flag',
  'srch_saturday_night_bool',
  'srch_query_affinity_score',
  'orig_destination_distance',
  'srch_person_per_room_score',
  'srch_adults_per_room_score',
  'delta_starrating',
  'visitor_hist_adr_usd_log',
  'price_usd_log',
  'has_purch_hist_bool',
  'has_historical_price',
  'year',
  'month',
  'day',
  'hour',
  'minute',
  'weekday _label0',
  'weekday _label1',
  'weekday _label2',
  'weekday _label3',
  'weekday _label4',
  'weekday _label5',
  'weekday _label6',
  'site_id_label0',
  'site_id_label1',
  'site_id_label2',
  'site_id_label3',
  'site_id_label4',
  'site_id_label5',
  'site_id_label6',
  'site_id_label7',
  'site_id_label8',
  'site_id_label9',
  'visitor_location_country_id_label0',
  'visitor_location_country_id_label1',
  'visitor_location_country_id_label2',
  'visitor_location_country_id_label3',
  'visitor_location_country_id_label4',
  'visitor_location_country_id

In [97]:
# change order
keys_property[0], keys_property[1] = keys_property[1], keys_property[0]
assert keys_property[0] == 'prop_id', 'incorrect order'
keys_property

['prop_id',
 'prop_review_score',
 'prop_brand_bool',
 'prop_location_score1',
 'prop_location_score2',
 'unavailable_comp',
 'available_comp',
 'avg_price_comp',
 'prop_country_id_label0',
 'prop_country_id_label1',
 'prop_country_id_label2',
 'prop_country_id_label3',
 'prop_country_id_label4',
 'prop_country_id_label5',
 'prop_country_id_label6',
 'prop_country_id_label7',
 'prop_country_id_label8',
 'prop_country_id_label9',
 'prop_starrating_label0',
 'prop_starrating_label1',
 'prop_starrating_label2',
 'prop_starrating_label3',
 'prop_starrating_label4',
 'prop_starrating_label5',
 'prop_log_historical_price_bin0',
 'prop_log_historical_price_bin1',
 'prop_log_historical_price_bin2',
 'prop_log_historical_price_bin3',
 'prop_log_historical_price_bin4',
 'click_bool',
 'booking_bool',
 'gross_bookings_usd',
 'random_bool',
 'score',
 'price_usd',
 'position',
 'travel_distance']

In [98]:
def extract_data(data, keys, k='srch_id'):
    data = data[keys]
#     data[k].unique()
    data_unique_rows = data.drop_duplicates(subset=k)
    assert data_unique_rows.shape[0] == data[k].unique().size, \
        'if key (srch_id) is equal, all non-property attributes should be equal as well'

    assert data_unique_rows[k].min() > 0, 'search id must be positive and nonzero'
    assert ~data_unique_rows[k].isna().any()
    assert ~data_unique_rows.isna().any().any()
    assert ~data_unique_rows.isin([np.nan, np.inf, -np.inf]).any().any()
    return data_unique_rows

def sample(data, keys, k='srch_id', rm_first_column=True):
    data_unique_rows = extract_data(data, keys, k)
    sample = data_unique_rows.sample(frac=1, random_state=seed)
    x_train = sample.values
    if rm_first_column:
        x_train = x_train[:,1:]
#         x_train = sample.values[:,1:,np.newaxis]    
    return x_train

In [99]:
# convert to np array & reshape
# data_unique_srch_id = extract_data(data, keys_search, 'srch_id')
# sample = data_unique_srch_id.sample(frac=1, random_state=seed)

# x_train = sample.values[:,1:,np.newaxis]
x_train = sample(data, keys_search, k='srch_id')

# x_train = data_unique_srch_id.values
# x_train = x_train.reshape(x_train.shape + (1,))
# y_labels = x_train[:,0,:]
# discretizer = preprocessing.KBinsDiscretizer(n_bins=y_labels.shape[0],
#          encode='onehot', strategy='uniform')
# y_train = discretizer.fit_transform(y_labels)
x_train.shape # , y_labels.shape, y_train.shape,

(40, 73)

In [100]:
assert ~np.any(np.isinf(x_train))
assert ~np.any(np.isinf(-x_train))
assert ~np.any(np.isnan(x_train))

In [101]:
x_train_users = x_train
keys_property = [k for k in keys_property if k != 'travel_distance']
x_train_items = sample(data, keys_property, k='prop_id')

In [102]:
x_train_users.shape, x_train_items.shape

((40, 73), (996, 35))

In [103]:
n_clusters = 10
models_user = {'KMeans': sklearn.cluster.KMeans(n_clusters, n_jobs=2, random_state=seed)
#                , 'Agglomerative': sklearn.cluster.AgglomerativeClustering(n_clusters)
             }
n_clusters = 10
models_item = {'KMeans': sklearn.cluster.KMeans(n_clusters, n_jobs=2, random_state=seed)
#                , 'Agglomerative': sklearn.cluster.AgglomerativeClustering(n_clusters)
             }

In [104]:
for k,model in models_user.items():
    model.fit(x_train_users)
for k,model in models_item.items():
    model.fit(x_train_items)

In [105]:
users = models_user['KMeans'].predict(x_train_users)

In [106]:
reload(util.data)
def user_func(row, srch_id):
    row_without_id = row[keys_search[1:]]
    return models_user['KMeans'].predict([row_without_id])[0]

def item_func(row, prop_id):
    row_without_id = row[keys_property[1:]]
    return models_item['KMeans'].predict([row_without_id])[0]

scores = util.data.scores_df(data, user_func, item_func)

In [107]:
# check minimal occurence
value_counts_user = scores.user.value_counts(ascending=True)
value_counts_item = scores.item.value_counts(ascending=True)
assert value_counts_user.iloc[0] > 2
assert value_counts_item.iloc[0] > 2

In [108]:
reader = Reader(rating_scale=(0,6))
data_scores = Dataset.load_from_df(scores, reader)
# surprise.model_selection.cross_validate(SVD(), data_scores, cv=5)

In [109]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = surprise.model_selection.GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data_scores)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.966769517834428
{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}


In [110]:
# model = SVD(**gs.best_params['rmse'])
model = SVD()

trainset, testset = surprise.model_selection.train_test_split(data_scores, test_size=0.3, random_state=seed)
model.fit(trainset)
predictions = model.test(testset)
surprise.accuracy.rmse(predictions), surprise.accuracy.fcp(predictions)

RMSE: 1.0264
FCP:  0.4645


(1.0263579533216072, 0.46449136276391556)

In [111]:
def predict(model, dataset):
    results = collections.defaultdict(dict)
    for item, user, score in testset:
        result = model.predict(str(item), str(user), verbose=0)
        results[user][item] = result.est
    return results
# surprise.SVD.predict()

In [112]:
scores_pred = predict(model, testset)
# scores_pred

In [113]:
# TODO use original user/item ids and convert results to csv
# i.e. 
result = collections.defaultdict(dict)
for row in 'train.csv':
    user_id = kmeans_model.predict(row[keys_search])
    item_id = kmeans_model.predict(row[keys_property])
    score = predict(model, (user_id, item_id))
    result[user_id][item_id] = score
    
# then convert result dict to df with columns (user id, prop id)

NameError: name 'kmeans_model' is not defined

In [ ]:
# surprise.model_selection.cross_validate(SVD(), data_scores, cv=5)

In [32]:
# y_train = y_train.toarray()
# y_train = y_train[:,:,np.newaxis]
# y_train.shape

In [17]:
# x_train = np.random.random(x_train.shape)
# y_train = np.random.random(y_train.shape)

In [16]:
# n_dims = data.shape[1]
input_shape = x_train.shape[1:]
output_shape = x_train.shape[1:]
print(input_shape, output_shape)

(35, 1) (35, 1)


In [33]:
encoder_input = Input(shape=input_shape)
h = encoder_input
h = Flatten()(h)
h = Dense(100, activation='relu')(h)
# decoder
h = Dense(np.prod(output_shape), activation='relu')(h)
h = Reshape(output_shape)(h)
encoder_output = Activation('sigmoid')(h)
model = Model(encoder_input, encoder_output, name='ae-')
model.compile(optimizer='adam', loss='binary_crossentropy') # RMSprop Adadelta SGD

(35, 1) (35, 1)


In [17]:
input_shape, output_shape

((35, 1), (35, 1))

In [36]:
# validation_split does not shuffle the data
result = model.fit(x_train, x_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# history = result.history

Train on 32 samples, validate on 8 samples
Epoch 1/2
32/32 [==============================] - 1s 33ms/step - loss: -35.1065 - val_loss: -38.4029
Epoch 2/2
32/32 [==============================] - 0s 433us/step - loss: -35.2843 - val_loss: -38.4029


In [37]:
# result.params
result.history

{'val_loss': [-38.40291213989258, -38.40291213989258],
 'loss': [-35.10652494430542, -35.28428649902344]}

In [22]:
# model.predict(x_train)

In [38]:
# vae, encoder, decoder = init_vae(input_shape, output_shape, n_latent_dims=10)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 35, 1)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 35)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               3600      
_________________________________________________________________
dense_10 (Dense)             (None, 35)                3535      
_________________________________________________________________
reshape_4 (Reshape)          (None, 35, 1)             0         
_________________________________________________________________
activation_2 (Activation)    (None, 35, 1)             0         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________


In [18]:
# keras.utils.plot_model(model, to_file='model.pdf', show_shapes=True, rankdir='TB') # TB LR

In [19]:
# First design an AE as simple as possible, to avoid overfitting
# I.e. reduce the number of nodes until the (train) performance drops
# Then increase the complexity (i.e. number of layers) to improve the compression

In [20]:
# see also
# https://github.com/voschezang/drum-style-transfer/blob/master/src/models.py
# https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py

# def vae_loss(vae_input: np.ndarray,
#              vae_output: np.ndarray,
#              z_mean: np.ndarray,
#              z_log_var: np.ndarray,
#              beta=0.,
#              extra_loss_func=keras.losses.mean_absolute_error,
#              gamma=0.):
#     """ Loss function for VAE
#     beta = disentanglement amount
#     gamma = multiplier for extra loss function
#     """
# #     vae_input = K.flatten(vae_input)
# #     vae_output = K.flatten(vae_output)
#     n_dims = np.prod(vae_input)
#     # bin. cross-entropy
#     xent_loss = n_dims * keras.metrics.binary_crossentropy(vae_input, vae_output)
#     # Kullback-Leibler divergence
#     kl_loss = -1 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     # (optional) kl_loss = max(kl_loss, vae_input)
#     extra_loss = n_dims * extra_loss_func(vae_input, vae_output)
#     vae_loss = K.mean(xent_loss + beta * kl_loss + gamma * extra_loss)
# #     vae_loss = kl_loss
#     vae_loss = xent_loss
# #     vae_loss = keras.losses.mean_absolute_error(vae_input, vae_output)
#     return vae_loss
def vae_loss(vae_input,
             vae_output,
             z_mean,
             z_log_var,
             timesteps=40,
             notes=9,
             beta=0.5,
             extra_loss_f=keras.losses.mean_absolute_error,
             gamma=0.5):
    """
    vae_input, vae_output == x == y :: np.ndarray
    z_mean, z_log_var :: np.ndarray
    beta = disentanglement amount
    gamma = multiplier for extra loss function
    """
    vae_input_ = K.flatten(vae_input)
    vae_output_ = K.flatten(vae_output)
    # bin. cross-entropy
    xent_loss = timesteps * notes * keras.metrics.binary_crossentropy(
        vae_input_, vae_output_)
    # Kullback-Leibler divergence
    kl_loss = -1. * K.sum(
        1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    # (optional) kl_loss = max(kl_loss, free_bits)
    extra_loss = timesteps * notes * extra_loss_f(vae_input_, vae_output_)

    # change beta
    #     beta = ((1.0 - tf.pow(hparams.beta_rate, tf.to_float(self.global_step)))
    #             * hparams.max_beta)
    #     self.loss = tf.reduce_mean(r_loss) + beta * tf.reduce_mean(kl_cost)
    # y_true, y_pred, z_mean, z_log_var, timesteps=150, notes=3, beta=1.

    vae_loss = K.mean(xent_loss + beta * kl_loss + gamma * extra_loss)
    return vae_loss


def init_encoder(input_shape=(10,1), n_latent_dims=2):
    encoder_input = Input(shape=input_shape)
    n_dims = np.prod(input_shape)
    
    # hidden layers
    h = encoder_input
    h = Flatten()(h)
    for i in range(1):
        h = Dense(100, activation='relu')(h)

    # Output = Distribution of latent space z (Mean, Variance)
    z_mean = Dense(n_latent_dims, name='z_mean')(h)  # no activation='relu'
    z_log_var = Dense(n_latent_dims, name='z_log_var')(h)  # no activation='relu'

    encoder_output = [z_mean, z_log_var]
    encoder = Model(encoder_input, encoder_output, name='encoder-')
    return encoder, encoder_input, z_mean, z_log_var


def init_decoder(output_shape=(10, 1), n_latent_dims=10):
    """ decoder_input = z_output
    decoder.predict :: z -> x
    the decoder models p(x|z), with an assumption over p(z)
    """
    # image_data_format = 'channels_last'
    n_dims = output_shape
    decoder_input = Input(shape=(n_latent_dims, ))
    # hidden layers
    h = decoder_input
    for i in range(1):
        h = Dense(100, activation='relu')(h)

    h = Dense(np.prod(output_shape), activation='relu')(h)
    h = Reshape(output_shape)(h)    
    return Model(decoder_input, h, name='decoder-')


def init_vae(input_shape=(10, 1), n_latent_dims=10, epsilon_std=1.):
    output_shape = input_shape
    # A classifier with the structure of a variational autoencoder (vae)
    print(input_shape, output_shape, n_latent_dims)
    encoder, encoder_input, z_mean, z_log_var = init_encoder(
        input_shape, n_latent_dims)
    z_input = encoder(encoder_input)
    print('z_input')
    
    # this function must be defined locally in order for the model to be serializable
    # using the following inline lambda function will result in an error when .to_json() is called
    def sample(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(
            shape=(K.shape(z_mean)[0], n_latent_dims),
            mean=0.,
            stddev=epsilon_std)
        return z_mean + K.exp(z_log_var) * epsilon

    z_output = Lambda(sample)(z_input)
    decoder = init_decoder(output_shape, n_latent_dims)
    # VAE
    vae_input = encoder_input
    vae_output = decoder(z_output)
    vae = Model(vae_input, vae_output, name='vae-')
    loss = vae_loss(vae_input, vae_output, z_mean, z_log_var)
#     loss = keras.losses.binary_crossentropy(vae_input, vae_output)
    vae.add_loss(loss)
    vae.compile(optimizer='adam') # binary_crossentropy
#     vae.compile(optimizer='adam', loss='binary_crossentropy')
    
    encoder_deterministic = Model(encoder_input, z_mean)
    return vae, encoder_deterministic, decoder

vae, encoder, decoder = init_vae(input_shape, n_latent_dims=10)

(35, 1) (35, 1) 10
z_input


In [21]:
# vae.summary()

In [22]:
# result = vae.fit(x_train, y_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
x = np.random.random(x_train.shape)
# result = vae.fit(x, x, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# result = vae.fit(x, x, epochs=2)
result = vae.fit(x_train, x_train, epochs=2)

ValueError: ('Error when checking model target: expected no data, but got:', array([[[17.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 0.]],

       [[29.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       [[31.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       ...,

       [[71.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 1.],
        [ 0.]],

       [[61.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 0.]],

       [[ 6.],
        [ 0.],
        [ 0.],
        ...,
        [ 0.],
        [ 0.],
        [ 1.]]]))

In [96]:
vae.predict(np.random.random(x_train.shape)[:2])[0,:5]

array([[ 0.3433886 ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.62094724]], dtype=float32)

In [ ]:
vae.predict(x_train[:2] * 0.3)[0,:5]

In [ ]:
np.any(np.isinf(x_train)), np.any(np.isnan(x_train))

In [48]:
z = encoder.predict(x_train[:2])
z.shape, z[0,:5]

((2, 10),
 array([-278.16318 ,  277.88397 , -123.55464 , -426.1635  ,  -53.177402],
       dtype=float32))

In [53]:
decoder.predict(z)[0,:5]

array([[ -0.        ],
       [ -0.        ],
       [  0.73194695],
       [ -0.        ],
       [127.27972   ]], dtype=float32)

https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
    
<img src="https://cdn-images-1.medium.com/max/1200/1*BrwllztzxZO89qCB9w5d-w.png" style="width:50%;"/>